# Using Trio CDP Inside Jupyter Notebook

Trio CDP can be used in Jupyter notebook, but some caveats are warranted. First, Trio support in Jupyter is experimental. Read [the instructions](https://github.com/ipython/ipykernel/pull/479) for setting up Trio inside Jupyter.

The second caveat is that Trio CDP uses [context variables](https://docs.python.org/3.7/library/contextvars.html?highlight=contextvar#contextvars.ContextVar) to keep track of which connection and/or session are associated with each task. In Jupyter notebook, however, each cell executes in a separate task, so connections and sessions are not automatically shared between cells. This notebook contains a workaround that allows a single connection and session to be shared across cells.

In [1]:
from trio_cdp import connect_cdp, dom, page, target

In [2]:
conn = await connect_cdp(GLOBAL_NURSERY,
    'ws://127.0.0.1:9000/devtools/browser/f01f56ba-993e-4c83-adc0-10a1feb56449')

In [3]:
# Here is where we have to do some magic to make the connection from the
# previous cell available to other cells.
import trio_cdp.context
trio_cdp.context.set_global_connection(conn)

In [4]:
# Now we can run Trio CDP commands and they execute on the connection
# automatically.
targets = await target.get_targets()

In [5]:
targets

[TargetInfo(target_id=TargetID('0AF1752B5834E8ED9F36D7BC3D1AEF51'), type='page', title='Hyperion Gray', url='https://www.hyperiongray.com/', attached=False, opener_id=None, browser_context_id=BrowserContextID('B2A138B23272D6E4920555C2DE424E05')),
 TargetInfo(target_id=TargetID('A0CF2CC043BDF2E4F8F8C0514B9A2FD2'), type='page', title='Hyperion Gray', url='https://www.hyperiongray.com/', attached=False, opener_id=None, browser_context_id=BrowserContextID('B2A138B23272D6E4920555C2DE424E05'))]

In [6]:
session = await conn.connect_session(targets[0].target_id)

In [7]:
# We have to do something similar for the session so that it can be
# reused across multiple cells.
trio_cdp.context.set_global_session(session)

In [8]:
async with session.page_enable():
    async with session.wait_for(page.LoadEventFired):
        await page.navigate('https://www.hyperiongray.com')

In [9]:
doc = await dom.get_document()

In [10]:
title = await dom.query_selector(doc.node_id, 'title')

In [11]:
await dom.get_outer_html(title)

'<title>Hyperion Gray</title>'